In [1]:
import wandb
import pandas as pd
import numpy as np
import texttable as tt
import latextable

In [8]:
EXPERIMENT_GROUP = "exp-1-dataset-split-strategy"
BASELINE_SETUP = "random"
AVERAGE_EPOCHS = 0 # 5 or 0 | 0 for no averaging, must be odd
HAS_WEIGHTED_ACCURACY = False
FIRST_N_METRICS = 6 #6 # Only weighted_accuracy and macro_f1 for the results table in experiment results section
EXCLUDED_SETUPS = [4,5] # [0, 1] | list of setups to exclude, by their index in the list of setups

In [3]:
def get_best_epoch_metrics(run, metric_name, average_epochs=0):
    history = pd.DataFrame(list(run.scan_history()))
    
    # Find the epoch with the best metric value
    best_epoch = history[metric_name].idxmax()
    best_metrics = history.loc[best_epoch]
    if average_epochs > 0:
        # Find indices of valid (non-NaN) values
        valid_indices = history[metric_name].dropna().index
        
        # Locate the best epoch index in the valid indices
        best_index_position = valid_indices.get_loc(best_epoch)
        
        # Get the indices of the average_epochs epochs to be averaged
        idx_adjustment = (average_epochs - 1) // 2
        start_index = max(0, best_index_position - idx_adjustment)
        end_index = min(len(valid_indices), best_index_position + idx_adjustment + 1)
        selected_indices = valid_indices[start_index:end_index]
        
        # Convert the selected indices to numeric, forcing errors to NaN - Use when averaging over epchs
        # converted_history = history.loc[selected_indices].apply(pd.to_numeric, errors='coerce')
        # converted_history.mean()

        # Average the metrics over the selected epochs
        averaged_metrics = history.loc[selected_indices].mean()
        
        print(f"Run {run.url} - Best epoch: {best_epoch} (averaged over {average_epochs}) - {metric_name}: {averaged_metrics[metric_name]}")
        return averaged_metrics, best_epoch
    else:
        print(f"Run {run.url} - Best epoch: {best_epoch} - {metric_name}: {best_metrics[metric_name]}")
        return best_metrics, best_epoch

def gather_metrics_for_experiment_group(experiment_group, average_epochs=0, has_weighted_accuracy=True):
    api = wandb.Api(timeout=60)

    # Fetch runs matching the experiment group
    runs = api.runs("crop-classification/messis", filters={"config.experiment_group": experiment_group})
    print(f"Found {len(runs)} runs for experiment group '{experiment_group}'")

    # Organize the runs by setup name
    setups = {}
    for run in runs:
        if run.state != 'finished':
            print(f"Run {run.url} - Skipping (not finished, state: {run.state})")
            continue
        if 'name' not in run.config:
            if 'experiment_name' in run.config:
                setup_name = run.config['experiment_name'].split('-')[-1]
                print(f"Run {run.url} - Missing 'name' in config, getting it from 'experiment_name' instead: {setup_name}")
            else:
                print(f"Run {run.url} - Missing 'name' in config, skipping")
                continue
        else:
            setup_name = run.config['name']
        if setup_name not in setups:
            setups[setup_name] = {
                'runs': [],
                'metrics': {metric: [] for metric in [
                    'val_f1_tier1_majority',
                    'val_f1_tier2_majority',
                    'val_f1_tier3_majority',
                    'val_weighted_accuracy_tier1_majority' if has_weighted_accuracy else 'val_accuracy_tier1_majority',
                    'val_weighted_accuracy_tier2_majority' if has_weighted_accuracy else 'val_accuracy_tier2_majority',
                    'val_weighted_accuracy_tier3_majority' if has_weighted_accuracy else 'val_accuracy_tier3_majority',
                    'val_precision_tier1_majority',
                    'val_precision_tier2_majority',
                    'val_precision_tier3_majority',
                    'val_recall_tier1_majority',
                    'val_recall_tier2_majority',
                    'val_recall_tier3_majority',
                    'val_cohen_kappa_tier1_majority',
                    'val_cohen_kappa_tier2_majority',
                    'val_cohen_kappa_tier3_majority',
                ]},
                'best_steps': []
            }
        setups[setup_name]['runs'].append(run)

    # Print setup names with number of runs
    for setup_name, setup_data in setups.items():
        print(f"Setup '{setup_name}' - {len(setup_data['runs'])} runs")

    # Gather metrics for each setup
    for setup_name, setup_data in setups.items():
        for run in setup_data['runs']:
            best_metrics, best_epoch = get_best_epoch_metrics(run, 'val_f1_tier3_majority' if (setup_name != 'seasonality' and setup_name != 'seasonality-reduced') else 'val_f1_tier2_majority', average_epochs)
            setup_data['best_steps'].append(best_epoch)
            for metric in setup_data['metrics']:
                # Skip metrics that are not present in the best_metrics
                if metric not in best_metrics:
                    print(f"Run {run.url} - Skipping metric '{metric}' (not present in run)")
                    continue
                setup_data['metrics'][metric].append(best_metrics[metric])
    
    # Prepare data for DataFrame
    data = []
    for setup_name, setup_data in setups.items():
        metrics_summary = {
            'setup': setup_name,
            'num_runs': len(setup_data['runs']),
            'average_epochs': average_epochs,
            'run_handles': [run.id for run in setup_data['runs']],
            'best_steps': setup_data['best_steps'],
        }
        for metric, values in setup_data['metrics'].items():
            metrics_summary[f'{metric}_mean'] = np.mean(values)
            metrics_summary[f'{metric}_std'] = np.std(values)
        data.append(metrics_summary)
    
    metrics_df = pd.DataFrame(data)
    
    return metrics_df

# Run the function for a given experiment group
metrics_df = gather_metrics_for_experiment_group(EXPERIMENT_GROUP, average_epochs=AVERAGE_EPOCHS, has_weighted_accuracy=HAS_WEIGHTED_ACCURACY)

# Save the DataFrame to a CSV file
metrics_df.to_csv(f"experiment_results/eval_{EXPERIMENT_GROUP}.csv", index=False)

metrics_df

Found 22 runs for experiment group 'exp-1-dataset-split-strategy'
Setup 'band' - 5 runs
Setup 'sechidis' - 5 runs
Setup 'random' - 5 runs
Setup 'stratified' - 5 runs
Setup 'stratified-longer' - 1 runs
Setup 'stratified-long' - 1 runs
Run https://wandb.ai/crop-classification/messis/runs/1pr54duf - Best epoch: 2108 - val_f1_tier3_majority: 0.1830819696187973
Run https://wandb.ai/crop-classification/messis/runs/0l8n9zsl - Best epoch: 1649 - val_f1_tier3_majority: 0.17700351774692535
Run https://wandb.ai/crop-classification/messis/runs/fo5way20 - Best epoch: 2093 - val_f1_tier3_majority: 0.17085738480091095
Run https://wandb.ai/crop-classification/messis/runs/10h585wi - Best epoch: 812 - val_f1_tier3_majority: 0.15809355676174164
Run https://wandb.ai/crop-classification/messis/runs/hc5nbhan - Best epoch: 2612 - val_f1_tier3_majority: 0.21968312561511993
Run https://wandb.ai/crop-classification/messis/runs/n0rdghiu - Best epoch: 3148 - val_f1_tier3_majority: 0.2041485458612442
Run https://w

,setup,num_runs,average_epochs,run_handles,best_steps,val_f1_tier1_majority_mean,val_f1_tier1_majority_std,val_f1_tier2_majority_mean,val_f1_tier2_majority_std,val_f1_tier3_majority_mean,...,val_recall_tier2_majority_mean,val_recall_tier2_majority_std,val_recall_tier3_majority_mean,val_recall_tier3_majority_std,val_cohen_kappa_tier1_majority_mean,val_cohen_kappa_tier1_majority_std,val_cohen_kappa_tier2_majority_mean,val_cohen_kappa_tier2_majority_std,val_cohen_kappa_tier3_majority_mean,val_cohen_kappa_tier3_majority_std
0,band,5,0,"[1pr54duf, 0l8n9zsl, fo5way20, 10h585wi, hc5nb...","[2108, 1649, 2093, 812, 2612]",0.429737,0.052202,0.308722,0.015791,0.181744,...,0.306548,0.020088,0.178099,0.019852,0.829278,0.036104,0.707195,0.036485,0.649716,0.033916
1,sechidis,5,0,"[n0rdghiu, 4ngwv6gg, h1gdnimy, 4dagsmy7, hugmr...","[3148, 2450, 2057, 1660, 2226]",0.441426,0.064666,0.294472,0.014107,0.182014,...,0.293794,0.010648,0.178544,0.009401,0.863307,0.016909,0.735517,0.007959,0.682186,0.009471
2,random,5,0,"[5gk5z23u, 5x6n23lh, 74xdec90, qurcrlyq, o1f3g...","[2894, 1781, 915, 2239, 2470]",0.441638,0.052631,0.287961,0.009391,0.171619,...,0.283082,0.014386,0.167847,0.016818,0.852319,0.011009,0.731308,0.017705,0.671994,0.023888
3,stratified,5,0,"[dszj9o60, y5u6yu1x, ji8uvg9q, 5o4qjb44, kbojj...","[1287, 1846, 952, 1809, 2591]",0.476175,0.077331,0.307169,0.020720,0.180253,...,0.297851,0.019110,0.172597,0.016597,0.853927,0.010182,0.716292,0.017909,0.660763,0.019636
4,stratified-longer,1,0,[islwn1g7],[7434],0.390121,0.000000,0.274251,0.000000,0.194545,...,0.260451,0.000000,0.185722,0.000000,0.868168,0.000000,0.725088,0.000000,0.679943,0.000000
5,stratified-long,1,0,[1wx9c8fy],[2064],0.389849,0.000000,0.268020,0.000000,0.148451,...,0.250327,0.000000,0.149984,0.000000,0.847390,0.000000,0.709557,0.000000,0.656246,0.000000


In [9]:
# Mapping for short metric names
metric_name_mapping = {
    'val_f1_tier1_majority': 'F1 $T_1$',
    'val_f1_tier2_majority': 'F1 $T_2$',
    'val_f1_tier3_majority': 'F1 $T_3$',
}

if HAS_WEIGHTED_ACCURACY:
    metric_name_mapping['val_weighted_accuracy_tier1_majority'] = 'W. Acc. $T_1$'
    metric_name_mapping['val_weighted_accuracy_tier2_majority'] = 'W. Acc. $T_2$'
    metric_name_mapping['val_weighted_accuracy_tier3_majority'] = 'W. Acc. $T_3$'
else:
    metric_name_mapping['val_accuracy_tier1_majority'] = 'Acc. $T_1$'
    metric_name_mapping['val_accuracy_tier2_majority'] = 'Acc. $T_2$'
    metric_name_mapping['val_accuracy_tier3_majority'] = 'Acc. $T_3$'

metric_name_mapping = { # Adding further metrics down here, to make sure we keep the most relevant metrics at the top
    **metric_name_mapping,
    'val_precision_tier1_majority': 'Precision $T_1$',
    'val_precision_tier2_majority': 'Precision $T_2$',
    'val_precision_tier3_majority': 'Precision $T_3$',
    'val_recall_tier1_majority': 'Recall $T_1$',
    'val_recall_tier2_majority': 'Recall $T_2$',
    'val_recall_tier3_majority': 'Recall $T_3$',
    'val_cohen_kappa_tier1_majority': 'Kappa $T_1$',
    'val_cohen_kappa_tier2_majority': 'Kappa $T_2$',
    'val_cohen_kappa_tier3_majority': 'Kappa $T_3$',
}

# Function to create a LaTeX table with bold best scores and underlined second-best scores
def create_latex_table(df, baseline_setup, first_n_metrics=None):
    metrics = list(metric_name_mapping.keys())
    if first_n_metrics:
        metrics = metrics[:first_n_metrics]
    
    # Determine the best and second-best scores for each metric
    best_scores = {metric: df[f'{metric}_mean'].max() for metric in metrics}
    second_best_scores = {
        metric: sorted(df[f'{metric}_mean'].unique(), reverse=True)[1] if len(df[f'{metric}_mean'].unique()) > 1 else None
        for metric in metrics
    }

    # Reorder the dataframe to have the baseline setup first
    df = df.set_index('setup')
    if baseline_setup in df.index:
        df = df.loc[[baseline_setup] + [idx for idx in df.index if idx != baseline_setup]].reset_index()
    else:
        df = df.reset_index()

    # Initialize the Texttable object
    table = tt.Texttable()
    
    # Define the header
    headers = ['Metric'] + [f"{'$\\dagger$ ' if row['setup'] == baseline_setup else ''}{str(setup).replace('_', '\\_')} ({row['num_runs']}-fold)" 
                            for idx, row in df.iterrows() for setup in [row['setup']]]
    table.header(headers)
    
    # Set alignment for columns
    cols_align = ["c"] * len(headers)
    cols_align[0] = "l"
    table.set_cols_align(cols_align)
    
    # Populate the table with data
    for metric in metrics:
        row_data = [metric_name_mapping[metric]]
        for idx, row in df.iterrows():
            mean = row[f'{metric}_mean'] * 100
            std = row[f'{metric}_std'] * 100
            value = f'{mean:.1f}\\% ± {std:.1f}\\%'
            if mean == best_scores[metric] * 100:
                value = f'\\textbf{{{value}}}'
            elif second_best_scores[metric] and mean == second_best_scores[metric] * 100:
                value = f'\\underline{{{value}}}'
            row_data.append(value)
        table.add_row(row_data)
    
    # Generate the LaTeX table
    average_epochs_used = df['average_epochs'].iloc[0] 
    caption_averaged =  "" if average_epochs_used == 0 else f" averaged over {average_epochs_used} epochs"
    latex_table = latextable.draw_latex(
        table, 
        caption=f"Results for experiment {EXPERIMENT_GROUP}. All metrics calculated on field majority during validation phase{caption_averaged}, with mean and std reported. Best scores bold, second best underlined, $\\dagger$ is baseline.", 
        label=f"tab:metrics_comparison_{EXPERIMENT_GROUP}", 
        use_booktabs=True,
        position="h"
    )

    # Add \resizebox to the LaTeX table
    latex_table = latex_table.replace('\\begin{center}', '\\begin{center}\\resizebox{\\textwidth}{!}{')
    latex_table = latex_table.replace('\\end{center}', '}\\end{center}')

    return latex_table

# Generate the LaTeX table
df = pd.read_csv(f"experiment_results/eval_{EXPERIMENT_GROUP}.csv")
df = df.sort_values(by='setup')

# Optional: Drop rows to exclude some setups
df = df.drop(EXCLUDED_SETUPS)

latex_table = create_latex_table(df, baseline_setup=BASELINE_SETUP, first_n_metrics=FIRST_N_METRICS)
print(latex_table)

\begin{table}[h]
	\begin{center}\resizebox{\textwidth}{!}{
		\begin{tabular}{lcccc}
			\toprule
			Metric & $\dagger$ random (5-fold) & band (5-fold) & sechidis (5-fold) & stratified (5-fold) \\
			\midrule
			F1 $T_1$ & \underline{44.2\% ± 5.3\%} & 43.0\% ± 5.2\% & 44.1\% ± 6.5\% & \textbf{47.6\% ± 7.7\%} \\
			F1 $T_2$ & 28.8\% ± 0.9\% & \textbf{30.9\% ± 1.6\%} & 29.4\% ± 1.4\% & \underline{30.7\% ± 2.1\%} \\
			F1 $T_3$ & 17.2\% ± 1.8\% & \underline{18.2\% ± 2.1\%} & \textbf{18.2\% ± 1.3\%} & 18.0\% ± 1.9\% \\
			Acc. $T_1$ & 44.2\% ± 5.9\% & 43.5\% ± 6.0\% & \underline{45.0\% ± 5.9\%} & \textbf{46.6\% ± 6.3\%} \\
			Acc. $T_2$ & 28.3\% ± 1.4\% & \textbf{30.7\% ± 2.0\%} & 29.4\% ± 1.1\% & \underline{29.8\% ± 1.9\%} \\
			Acc. $T_3$ & 16.8\% ± 1.7\% & \underline{17.8\% ± 2.0\%} & \textbf{17.9\% ± 0.9\%} & 17.3\% ± 1.7\% \\
			\bottomrule
		\end{tabular}
	}\end{center}
	\caption{Results for experiment exp-1-dataset-split-strategy. All metrics calculated on field majority during valida

In [5]:
# deserialize best_steps column in df to list
df['best_steps'] = df['best_steps'].apply(eval)

In [6]:
# calculate mean and std for each list in column best_steps
df['best_steps_mean'] = df['best_steps'].apply(np.mean)
df['best_steps_std'] = df['best_steps'].apply(np.std)

In [7]:
# show columns setup, best_steps_mean and num_runs
df[['setup', 'best_steps_mean', 'best_steps_std', 'num_runs']]

,setup,best_steps_mean,best_steps_std,num_runs
0,band,1854.8,603.984569,5
2,random,2059.8,675.953075,5
1,sechidis,2308.2,493.029979,5
3,stratified,1697.0,558.051252,5
5,stratified-long,2064.0,0.000000,1
4,stratified-longer,7434.0,0.000000,1
